### Модуль plotly 2. Домашнее задание

Все графики должны быть построены с помощью **plotly.express**.

#### 1. Анализ акций Tesla

Скачайте **yahoo-finance** и библиотеку **pandas_datareader** для загрузки данных.

In [87]:
import plotly.express as px
from datetime import datetime
import datetime as dt
import pandas_datareader.data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

In [88]:
# startdate = datetime(2020,1,1)
# enddate = datetime(2020,12,21)
startdate = '2020-1-1'
enddate = '2020-12-1'
data = pdr.get_data_yahoo(['TSLA'], start=startdate, end=enddate)
data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-23,167.833328,175.333328,167.263336,173.949997,173.949997,150780900
2020-11-24,180.133331,186.663330,175.399994,185.126663,185.126663,160945500
2020-11-25,183.353333,191.333328,181.789993,191.333328,191.333328,146790600
2020-11-27,193.720001,199.593338,192.816666,195.253326,195.253326,112683300
2020-11-30,200.736664,202.600006,184.836670,189.199997,189.199997,189009300


**1.a.** Постройте две линии на одном графике: изменение самой высокой цены High и самой низкой Low в течение 2020 года.

Напишите месяц, в котором самая высокая цена акции за день впервые превысила 350 долларов за штуку.

**Ваш ответ:** Нет такого месяца где стоимость првысила 350 за акцию (Максимальная 202)

In [89]:
data[['High', 'Low']].max()

High    202.600006
Low     192.816666
dtype: float64

In [90]:
fig = px.line(data_frame=data, x=data.index, y=['Low', 'High'])
fig.show()

**1.b.** Постройте следующий график:

- процентое изменение цены за акцию Tesla за день по отношению к цене на открытии торгов.

На оси **x** должны быть отмечены даты из data.index. Значения оси **y** должны отражать, на сколько процентов увеличилась (положительное число) или на сколько процентов упала (отрицательное число) цена акции на закрытии Close по сравнению с ценой открытия Open.

В какой день наблюдалось самое резкое понижение цены за акцию? А самое большое повышение? На сколько процентов?

*Подсказка: создайте новую колонку Diff, в которой посчитайте изменение акции за день в процентах*

**Ваш ответ: max diff = 15.78, min diff = -10.76**

In [91]:
data['Diff'] = (data.Close - data.Open) / data.Open * 100
data[data['Diff'] > 15][['Open', 'Close', 'Diff']]

,Open,Close,Diff
Date,,,
2020-02-03,44.912666,52.0,15.780256


In [92]:
data[['Open', 'Close', 'Diff']].min()

Open     24.980000
Close    24.081333
Diff    -10.757232
dtype: float64

**1.с.** С помощью **pandas** преобразуйте индекс датасета в формат «год-месяц» и сгруппируйте данные так, чтобы новому индексу соответствовали средние значения по колонкам в указанном месяце и годе.

Напишите месяц, когда объем акций впервые превысил 60M. Какая средняя цена акции на закрытии торгов соответствует этому месяцу?

*Подсказка: используйте параметр hover_data.*

**Ваш ответ:**

In [105]:
data_2 = data.reset_index()
data_2 = data_2.set_index(pd.to_datetime(data_2['Date']).dt.strftime('%m/%Y'))
del(data_2['Date'])

In [106]:
data_2 = data_2.groupby(data_2.index).mean()

In [110]:
fig2 = px.line(data_2, x=data_2.index, y='Open', hover_data=data_2.columns)
fig2.show()

#### 2. Анализ продаж мобильных телефонов

В этой части будем анализировать данные из соревнования Mobile Price Classification на Kaggle:

*battery_power: Total energy a battery can store in one time measured in mAh*

*dual_sim: Has dual sim support or not*

*fc: Front Camera mega pixels*

*four_g: Has 4G or not*

*int_memory: Internal Memory in Gigabytes*

*mobile_wt: Weight of mobile phone*

*n_cores: Number of cores of processor*

*pc: Primary Camera mega pixels*

*ram: Random Access Memory in Mega Bytes*

*sc_h: Screen Height of mobile in cm*

*sc_w: Screen Width of mobile in cm*

*talk_time: longest time that a single battery charge will last when you are*

*price_range: This is the target variable with value of 0(low cost), 1(medium cost), 2(high cost) and 3(very high cost).*

In [135]:
data = pd.read_csv('mobile_data.csv')

In [132]:
data.head()

,battery_power,dual_sim,fc,four_g,int_memory,mobile_wt,n_cores,pc,ram,sc_h,sc_w,talk_time,price_range
0,1954,1,0,0,24,187,4,0,700,16,3,5,0
1,1520,0,5,1,33,177,8,18,3826,14,9,13,3
2,775,0,3,0,46,159,2,16,568,17,15,11,0
3,961,1,0,1,57,114,8,3,2782,18,9,7,2
4,1453,1,12,1,52,96,2,18,2373,10,1,10,2


**1.a.** Загрузите данные. Постройте график scatter зависимости параметра ram от battery_power. С помощью одного графика ответьте на следующие вопросы:

- Существует ли зависимость ram от мощности батареи battery power?
- Правда ли, что чем больше значение battery_power, тем дороже будет стоить телефон?
- Верно ли предыдущее утверждение для параметра ram?
- Проверьте гипотезу, что у телефона с мощной батареей и большим значением ram время разговора (talk_time) тоже увеличено.

*Подсказка: переведите значения price_range в строки для того, чтобы легенда корректно отображалась;
используйте параметры color, size*.

**Ваш ответ:**

- Существует ли зависимость ram от мощности батареи battery power? **нет**
- Правда ли, что чем больше значение battery_power, тем дороже будет стоить телефон?
**нет**
- Верно ли предыдущее утверждение для параметра ram?
**да**
- Проверьте гипотезу, что у телефона с мощной батареей и большим значением ram время разговора (talk_time) тоже увеличено.
**нет кореляции**

In [140]:
data['price_range'] = data['price_range'].apply(str)

In [146]:
paper = px.scatter(data, x='battery_power', y='ram', color='price_range', trendline_scope='overall', trendline = 'ols', size='talk_time')
paper.show()

**2.b.** Изучите параметр **facet_col** и постройте зависимость памяти ram от количества мегапикселей камеры pc. У вас должно получиться четыре графика в зависимости от значения price_range.

In [152]:
paper_2 = px.scatter(data, x='ram', y='pc', color='price_range', facet_col='price_range', height=400)
paper_2.show()